In [1]:
import logging, sys
logging.disable(sys.maxsize)
import warnings
warnings.filterwarnings('ignore')

from types import SimpleNamespace
from extraction import *
import scipy.sparse as sp
from aind_ophys_utils.summary_images import pnr_image
from multiprocessing.pool import Pool

2025-01-27 19:10:35.658550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
args = SimpleNamespace(
        input_dir="../data/",
        output_dir="../results/",
        tmp_dir="/scratch",
        diameter=0,
        anatomical_only=2,
        denoise=False,
        cellprob_threshold=0.0,
        flow_threshold=1.5,
        spatial_hp_cp=0,
        pretrained_model="cyto",
        neuropil="CNMF",
        rf=100, stride=20, K=20, merge_thr=.6, ssub=2, tsub=2, normalize_init=True, init='greedy_roi',
        contour_video=False,
    )

In [3]:
    output_dir = Path(args.output_dir).resolve()
    input_dir = Path(args.input_dir).resolve()
    tmp_dir = Path(args.tmp_dir).resolve()
    session, data_description, subject = get_metdata(input_dir)
    subject_id = subject.get("subject_id", "")
    name = data_description.get("name", "")
    setup_logging("aind-ophys-extraction-suite2p", mouse_id=subject_id, session_name=name)
    if next(input_dir.rglob("*decrosstalk.h5"), ""):
        input_fn = next(input_dir.rglob("*decrosstalk.h5"))
    else:
        input_fn = next(input_dir.rglob("*registered.h5"))
    parent_directory = input_fn.parent
    if session is not None and "Bergamo" in session["rig_id"]:
        motion_corrected_fn = bergamo_segmentation(input_fn, session, temp_dir=tmp_dir)
    else:
        motion_corrected_fn = input_fn
    if not data_description or "multiplane" in data_description.get("name", ""):
        unique_id = motion_corrected_fn.parent.parent.name
    else:
        unique_id = "_".join(str(data_description["name"]).split("_")[-3:])

    frame_rate = get_frame_rate(session)

    output_dir = make_output_directory(output_dir, unique_id)

In [4]:
def format_caiman_output(e, cnmfe):
    assert np.allclose(linalg.norm(e.A, 2, 0), 1)
    traces_corrected = (e.C + e.YrA).astype("f4")
    if cnmfe: 
        traces_corrected += e.A.T.dot(e.b0)[:, None]
        e.b, e.fs = None, None  # required patch for next line
        traces_neuropil = e.A.astype("f4").T.dot(e.compute_background(caiman.movie(movie).to2DPixelxTime()))
    else:
        traces_neuropil = e.A.T.dot(e.b).dot(e.f).astype("f4")
        traces_corrected += .8 * traces_neuropil
    traces_roi = (e.C + e.YrA + traces_neuropil).astype("f4")
    # convert ROIs to sparse COO 3D-tensor  a la https://sparse.pydata.org/en/stable/construct.html
    data = []
    coords = []
    for i in range(e.A.shape[1]):
        roi = coo_matrix(e.A[:,i].reshape(e.dims, order="F").toarray(), dtype="f4")
        data.append(roi.data)
        coords.append(np.array([i*np.ones(len(roi.data)), roi.row, roi.col], dtype="i2"))
    if len(data):
        data = np.concatenate(data)
        coords = np.hstack(coords)  
    # TODO: save background, use caiman's classifier for iscell
    neuropil_coords, iscell, keys = [[]] * 3
    return traces_corrected, traces_neuropil, traces_roi, data, coords, neuropil_coords, iscell, keys 

In [5]:
    # def run_CaImAn(input_fn, unique_id, args):
    os.environ['CAIMAN_TEMP'] = args.tmp_dir
    cnmfe = args.neuropil.lower() == "cnmf-e"
    
    # create mmap file
    with h5py.File(input_fn) as fin:
        data = fin["data"]
        if data.nbytes < 1e9:
            fname_new = caiman.save_memmap([str(input_fn)], var_name_hdf5='data',
                                           order='C', base_name=unique_id)
        else:
            chunksize = 50
            chunkfiles = [args.tmp_dir + f"/chunk{k}.h5" for k in range(0, data.shape[0], chunksize)]
            def write_chunk(k):
                with h5py.File(args.tmp_dir + f"/chunk{k}.h5", "w") as f:
                    f.create_dataset('data', data=data[k:k+chunksize])
            with Pool() as pool:
                pool.map(write_chunk, range(0, data.shape[0],chunksize))
                fname_new = caiman.save_memmap(chunkfiles, var_name_hdf5='data',
                                           order='C', dview=pool, base_name=unique_id, n_chunks=16)
                pool.map(os.remove, chunkfiles)  # remove temporary files       
        
    # now load the file
    Yr, dims, T = caiman.load_memmap(fname_new)
    images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    # Set params
    gSig = args.diameter / 2
    params_dict={'fnames': fname_new,
                'K': args.K,
                'p': 1,
                'nb': 0 if cnmfe else 1,
                'rf': args.rf,
                'stride': args.stride,
                'only_init': cnmfe,
                'gSig': (gSig, gSig),
                'ssub': args.ssub,
                'tsub': args.tsub,
                'merge_thr': args.merge_thr,
                'method_init': args.init,
                'normalize_init': args.normalize_init}
    opts = params.CNMFParams(params_dict=params_dict)
    n_jobs = tmp if (tmp:=os.environ.get("CO_CPUS")) is None else int(tmp)
    with Pool(n_jobs) as pool:
        cnm = cnmf.CNMF(n_processes=pool._processes, params=opts, dview=pool)
        cnm.fit(images)
    traces_corrected, traces_neuropil, traces_roi, data, coords, neuropil_coords, iscell, keys = format_caiman_output(cnm.estimates, cnmfe)

The number of pixels per process (n_pixels_per_process) is larger than the total number of pixels!! Decreasing suitably.
The number of pixels per process (n_pixels_per_process) is larger than the total number of pixels!! Decreasing suitably.
